<a href="https://colab.research.google.com/github/klarcika/TZ/blob/main/Naloga9_Navodila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pridobivanje podatkov iz spletnih strani

In [2]:
!pip install Scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.5 MB/s eta 0:00:00


In [ ]:
import scrapy
import re #odstranimo presledke, nezelene vrstice, tabulatorji

class StudentskaDelaSpider(scrapy.Spider):
  name="tudentska_dela_spider"#ce delamo lokalno bomo na to ime ustvarili nov projekt
  #dolocimo na katere domene ga bomo spustili
  allowed_domains="studentski-servis.com" #lahko napisemo vec domen ["","",...]
  start_urls=["https://www.studentski-servis.com/studenti/prosta-dela/"]
  #obicajno iscemo neke sezname, da so podatki ze v struktorirani obliki
  custom_settings={ #kam se nam bojo  podatki shranili
      'FEEDS': {
          'dela.csv':{
              'format':'csv',
              'overwrite': True #vsakic ko kaj novega napisemo da jo na novo napise/ posodobi
          }
      }
  }

def parse(self, res):
  vsa_dela=res.css("article.job-item") #iskanje ele, ki ga zelimo pridobit; kako je posamezno delo zapisano, xpath ali css (inspect)
  #pajek mora priti samo do unikatinega identifikatorja
  d={} #slovar naslo, opis
  for delo in vsa_dela:
    d["naziv"]= delo.css("article.job-item h5.mb-0::text").extract_first() #prvi stolpec v potem shranjeni dat. csv
                          #da extractamo samo text zapisemo ::text
    lokacija=delo.css("article.job-item p::text").extract()[1] #začasna spremenljivka, ce iammo podatek vmes ga samo dolocimo z []
    d["lokacija"]=re.sub(r"[\n\t]*","",lokacija) #re.sub uporabimo za odstranjevanje
    yield d;

#to kodo smo napisali samo za 1 stran

  naslednje_strani= res.css("div.page-items s.page-link::text").extract() #dostopamo do št. strani ki obstajajo, da pridobimo vse podatke
                                                             #extractamo cisto vse
  for naslednja in naslednje_strani:
    if (int(naslednja)<3):
      url_str="https://www.studentski-servis.com/studenti/prosta-dela/?page"+naslednja
      yield scrapy.Request(url_str, callback=self.parse)
#gremo na stran ki smo jo dolicili na url-ju

In [ ]:
from scrapy.crawler import CrawlerProcess

proces= CrawlerProcess()
proces.crawl(StudentskaDelaSpider)

#če kaj spreminjamo moramo vse skupaj še enkrat zagnat